## Compiling into Booleans and Calling SAT-solver

In [1]:
import sys
import time
from ortools.sat.python import cp_model

In [10]:
model = cp_model.CpModel()

In [11]:
board_size=int(input())

In [12]:
# There are `board_size` number of variables, one for a queen in each column
# of the board. The value of each variable is the row that the queen is in.
queens=[]
for i in range(board_size):
    queens.append([])
    for j in range(board_size):
        queens[i].append(model.new_bool_var(f"x_{j}_{i}"))

display(queens)

[[x_0_0(0..1), x_1_0(0..1), x_2_0(0..1), x_3_0(0..1)],
 [x_0_1(0..1), x_1_1(0..1), x_2_1(0..1), x_3_1(0..1)],
 [x_0_2(0..1), x_1_2(0..1), x_2_2(0..1), x_3_2(0..1)],
 [x_0_3(0..1), x_1_3(0..1), x_2_3(0..1), x_3_3(0..1)]]

In [13]:
# All rows must be different.
for i in range(board_size):
    model.add_at_most_one(queens[j][i] for j in range(board_size))
    model.add_at_least_one(queens[j][i] for j in range(board_size))

# No two queens can be on the same diagonal.
for j in range(board_size-1):
    model.add_at_most_one(queens[j][j+1])
for j in range(1,board_size):
    model.add_at_most_one(queens[j][j-1])

In [14]:
class NQueenSolutionPrinter(cp_model.CpSolverSolutionCallback):
    """Print intermediate solutions."""

    def __init__(self, queens: list[list[cp_model.BoolVarT]]):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.__queens = queens
        self.__solution_count = 0
        self.__start_time = time.time()

    @property
    def solution_count(self) -> int:
        return self.__solution_count

    def on_solution_callback(self):
        current_time = time.time()
        print(
            f"Solution {self.__solution_count}, "
            f"time = {current_time - self.__start_time} s"
        )
        self.__solution_count += 1

        all_queens = range(len(self.__queens))
        for i in all_queens:
            for j in all_queens:
                if self.value(self.__queens[j][i]):
                    # There is a queen in column j, row i.
                    print("Q", end=" ")
                else:
                    print("_", end=" ")
            print()
        print()

In [15]:
# Solve the model.
solver = cp_model.CpSolver()
solution_printer = NQueenSolutionPrinter(queens)
solver.parameters.enumerate_all_solutions = True
solver.solve(model, solution_printer)

Solution 0, time = 0.0008306503295898438 s
_ Q _ _ 
Q _ _ _ 
Q _ _ _ 
Q _ _ _ 

Solution 1, time = 0.0010704994201660156 s
_ Q _ _ 
_ Q _ _ 
Q _ _ _ 
Q _ _ _ 

Solution 2, time = 0.00115966796875 s
_ Q _ _ 
_ Q _ _ 
_ Q _ _ 
Q _ _ _ 

Solution 3, time = 0.0012371540069580078 s
_ Q _ _ 
Q _ _ _ 
_ Q _ _ 
Q _ _ _ 

Solution 4, time = 0.0013113021850585938 s
_ Q _ _ 
Q _ _ _ 
_ Q _ _ 
_ Q _ _ 

Solution 5, time = 0.001384735107421875 s
_ Q _ _ 
_ Q _ _ 
_ Q _ _ 
_ Q _ _ 

Solution 6, time = 0.00145721435546875 s
_ Q _ _ 
_ Q _ _ 
Q _ _ _ 
_ Q _ _ 

Solution 7, time = 0.0015347003936767578 s
_ Q _ _ 
Q _ _ _ 
Q _ _ _ 
_ Q _ _ 

Solution 8, time = 0.0016062259674072266 s
_ _ Q _ 
Q _ _ _ 
Q _ _ _ 
_ Q _ _ 

Solution 9, time = 0.0016798973083496094 s
_ _ Q _ 
_ Q _ _ 
Q _ _ _ 
_ Q _ _ 

Solution 10, time = 0.001752614974975586 s
_ _ Q _ 
_ Q _ _ 
_ Q _ _ 
_ Q _ _ 

Solution 11, time = 0.0018236637115478516 s
_ _ Q _ 
Q _ _ _ 
_ Q _ _ 
_ Q _ _ 

Solution 12, time = 0.0018949508666992188 s
_ _

4

In [37]:
# Statistics.
print("\nStatistics")
print(f"  conflicts      : {solver.num_conflicts}")
print(f"  branches       : {solver.num_branches}")
print(f"  wall time      : {solver.wall_time} s")
print(f"  solutions found: {solution_printer.solution_count}")


Statistics
  conflicts      : 0
  branches       : 0
  wall time      : 0.000146756 s
  solutions found: 0


## Constraint Problem Solver

In [38]:
import sys
from ortools.constraint_solver import pywrapcp

In [39]:
solver = pywrapcp.Solver("n-queens")

In [ ]:
board_size=int(input())

In [40]:
# The array index is the column, and the value is the row.
queens = [solver.IntVar(0, board_size - 1, f"x{i}") for i in range(board_size)]

In [41]:
# All rows must be different.
solver.Add(solver.AllDifferent(queens))

# No two queens can be on the same diagonal.
solver.Add(solver.AllDifferent([queens[i] + i for i in range(board_size)]))
solver.Add(solver.AllDifferent([queens[i] - i for i in range(board_size)]))

In [42]:
db = solver.Phase(queens, solver.CHOOSE_FIRST_UNBOUND, solver.ASSIGN_MIN_VALUE)

In [43]:
# Iterates through the solutions, displaying each.
num_solutions = 0
solver.NewSearch(db)
while solver.NextSolution():
        # Displays the solution just computed.
    for i in range(board_size):
        for j in range(board_size):
            if queens[j].Value() == i:
                    # There is a queen in column j, row i.
                print("Q", end=" ")
            else:
                print("_", end=" ")
        print()
    print()
    num_solutions += 1
solver.EndSearch()

_ _ Q _ 
Q _ _ _ 
_ _ _ Q 
_ Q _ _ 

_ Q _ _ 
_ _ _ Q 
Q _ _ _ 
_ _ Q _ 



In [44]:
 # Statistics.
print("\nStatistics")
print(f"  failures: {solver.Failures()}")
print(f"  branches: {solver.Branches()}")
print(f"  wall time: {solver.WallTime()} ms")
print(f"  Solutions found: {num_solutions}")



Statistics
  failures: 4
  branches: 10
  wall time: 113721 ms
  Solutions found: 2
